## Load Data

In [1]:
import sys
sys.path.append("../")

from federated_gbdt.models.gbdt.private_gbdt import PrivateGBDT
from experiments.experiment_helpers.data_loader import DataLoader
from sklearn.metrics import roc_auc_score

In [2]:
dataloader = DataLoader()

# Default is 70/30 split
X_train, X_test, y_train, y_test = dataloader.load_datasets(["adult"], return_dict=False)[0]

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')
Counter({0: 24720, 1: 7841})
[Data Loader] Adult Class Balance: 0.2408095574460244 



In [3]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
4245,10,0,21194,11,8,4,2,1,2,1,0,51,39,38
22765,43,5,3536,11,8,2,11,0,4,1,0,0,44,38
9688,5,6,11237,2,7,4,3,1,4,1,0,0,49,38
18653,18,3,15683,15,9,2,11,0,4,1,0,0,42,38
1351,15,4,12714,9,12,2,2,0,4,1,94,0,49,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24286,38,3,8098,0,5,2,7,0,2,1,38,0,39,38
30626,18,3,2161,9,12,2,3,0,4,1,94,0,49,38
29914,16,3,21435,9,12,5,3,4,2,0,0,0,39,38
31025,44,3,4386,11,8,2,2,0,4,1,0,0,39,38


## XGBoost Training (No DP)

In [4]:
xgb_model = PrivateGBDT(num_trees=100, epsilon=0)
xgb_model = xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.9223165503224346

## DP-XGBoost (FEVERLESS)

In [5]:
dp_xgb_model = PrivateGBDT(num_trees=100, epsilon=1, dp_method="gaussian_cdp")
dp_xgb_model = dp_xgb_model.fit(X_train, y_train)
y_pred = dp_xgb_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.7968121875283752

## DP-TR XGBoost

In [6]:
dp_tr_model = PrivateGBDT(num_trees=100, epsilon=1, split_method="totally_random", 
                           sketch_type="uniform", dp_method="gaussian_cdp")

dp_tr_model = dp_tr_model.fit(X_train, y_train)
y_pred = dp_tr_model.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred)

0.8776826643425335